In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager 


In [9]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
url = "https://www.worten.pt/"
driver.get(url)
driver.implicitly_wait(5) # Not the best approach

In [10]:
# So we got a cookie pop-up, let's accept it to get it out of the way
cookies_modal = driver.find_element(By.CSS_SELECTOR,"div.modal__content")
accept_button = cookies_modal.find_element(By.XPATH,".//button[span[normalize-space()='Aceitar cookies']]").click()

In [11]:
to_search = "iphone 16"

# Let's fill the search box with the search term
# Find the search box
search_textbox = driver.find_element(By.ID, 'search')
# search_textbox = driver.find_element(By.CSS_SELECTOR, '#search')

# Send the text to the search box
search_textbox.send_keys(to_search)

# Find the search button and click it
# driver.find_element(By.XPATH, "//button[@aria-label='Pesquisar']").click()

# Another way just to show more possibilities
# You know the search textbox is inside a div with BUTTON TEXTBOX BUTTON, and the search button is the last element
# So you can fetch the textbox parent and then the last button
# parent_element = search_textbox.find_element(By.XPATH, '..')
# search_button = parent_element.find_elements(By.TAG_NAME, 'button')[-1]
# search_button.click()


# Problem is, worten's site changes the search button to a delete button when you type something.
# So, let's just send the enter key
from selenium.webdriver.common.keys import Keys
search_textbox.send_keys(Keys.RETURN)

driver.implicitly_wait(10) # Not the best approach


In [14]:
# Let's now get the results
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

# #filtered-grid > div > div > section > div > div > ul > li:nth-child(2)

# #filtered-grid > div > div > section > div > div > ul > li:nth-child(2) > div > a > div
product_items = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#filtered-grid ul li div.product-card__content')))
items_parsed = []
# Step 2: Iterate over each product item and extract name and price
for item in product_items:
    try:
        # Extract the product name
        name_element = item.find_element(By.CSS_SELECTOR, "h3.product-card__name span.produc-card__name__link")
        product_name = name_element.text.strip()

        # Extract the product price from the meta tag
        price_meta = item.find_element(By.CSS_SELECTOR, "meta[itemprop='price']")
        product_price = price_meta.get_attribute('content')
        # Parse the price str to float
        product_price = float(product_price)


        # Alternatively, extract the displayed price text
        # price_element = item.find_element(By.CSS_SELECTOR, "span.product-card__price")
        # product_price = price_element.text.strip()
        # product_price = float(product_price.replace('€','').replace('.','').replace(',','.').replace('\n',''))

        # Append the extracted data to the list
        items_parsed.append({
            'name': product_name,
            'price': product_price
        })
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
(items_parsed)

In [ ]:
# We have fetched the first page, but how do we go to the next page?

#  #filtered-grid > div > div > section > div > div > div.listing-content__pagination > div > div > div:nth-child(4)
next_button = driver.find_element(By.CSS_SELECTOR, "#filtered-grid > div > div > section > div > div > div.listing-content__pagination > div > div > div:nth-child(4)")
next_button.click()

In [17]:
driver.quit()